<a href="https://colab.research.google.com/github/FadelYang/craft-it-object-detection-model/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#importing required Libraries
import os
import cv2

# Preparing Dataset

## Mount to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
%cd '/content/drive/MyDrive/Colab NoteBooks/'
!pwd

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab NoteBooks/'
/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab Notebooks


## Connect To Kaggle And Download Datasets

this code is used to connect Google Colab and Kaggle, run it just once for download the datasets from kaggle, uncomment it first if you want to run these cell below

In [18]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
!mkdir /.kaggle
!mv kaggle.json /.kaggle
!mv /.kaggle /root/
!chmod 600 ~/.kaggle/kaggle.json

In [19]:
# Download datasets from Kaggle.
!kaggle datasets download -d kshitizgajurel042/recyclable-images-with-annotations-for-detection

Dataset URL: https://www.kaggle.com/datasets/kshitizgajurel042/recyclable-images-with-annotations-for-detection
License(s): unknown
 99% 511M/514M [00:06<00:00, 78.8MB/s]
100% 514M/514M [00:06<00:00, 88.1MB/s]


In [ ]:
!unzip recyclable-images-with-annotations-for-detection.zip

In [21]:
!mv 'Final Data' recyclable-images-datasets

## Inspect Dataset

In [22]:
# Define function for read annotations
def read_annotations(annotation_file):
  with open(annotation_file, 'r') as file:
    lines = file.readlines()
    annotations = []
    for line in lines:
      parts = line.strip().split()
      class_label = parts[0]
      x_min, y_min, x_max, y_max = map(float, parts[1:])
      annotations.append({
          'class_label': class_label,
          'x_min': x_min,
          'y_min': y_min,
          'x_max': x_max,
          'y_max': y_max
      })
    return annotations

In [26]:
# Preprocessing data
data_dir = os.path.join(os.getcwd(), 'recyclable-images-datasets')

def preprocess_data(data_dir):
  images = []
  annotations = []

  image_folder = os.path.join(data_dir, 'images', 'train')
  label_folder = os.path.join(data_dir, 'labels', 'train')

  for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
      image_path = os.path.join(image_folder, image_filename)
      annotation_filename = image_filename.replace('.jpg', '.txt').replace('.png', '.txt')
      annotation_path = os.path.join(label_folder, annotation_filename)

      image = cv2.imread(image_path)

      # Resize image.
      image = cv2.resize(image, (416, 416))

      # Normalize image
      image = image/255.0

      if os.path.exists(annotation_path):
        annotation = read_annotations(annotation_path)
        images.append(image)
        annotations.append(annotation)
      else:
        print(f'Annotation file for image {image_filename} not found')

  return images, annotations

In [29]:
images, annotations = preprocess_data(data_dir)